In [1]:
import re as regex
import pandas as pd
from data_extraction import DataExtractor

In [2]:
products_df = DataExtractor.extract_from_s3('s3://data-handling-public/products.csv')

In [5]:
products_df.head(5)

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6kg,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,Tiffany's World Day Out At The Park,£12.99,0.48kg,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,Tiffany's World Pups Picnic Playset,£7.00,590g,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,Tiffany's World Wildlife Park Adventures,£12.99,540g,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,Cosatto Cosy Dolls Pram,£30.00,1.91kg,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a


In [16]:
products_df[products_df['date_added'].str.contains('[A-Za-z ]+', regex=True, na=False)]

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
307,Pets Pantry Complete Meaty Chunks Beef 2.75kg,£3.69,2.75kg,pets,4525436759460,2018 October 22,6fc67918-7940-4b1f-bba4-47f3e6dfc578,Still_avaliable,l1-2836416D
751,VLPCU81M30,XCD69KUI0K,9GO9NZ5JTL,S1YB74MLMJ,OO7KH8P79I,CCAVRB79VV,7QB0Z9EW1G,T3QRRH7SRP,SDAV678FVD
1133,9SX4G65YUX,N9D2BZQX63,Z8ZTDGUZVU,C3NCA2CL35,E8EOGWOY8S,09KREHTMWL,CP8XYQVGGU,BPSADIOQOK,BSDTR67VD90
1217,Black Ceramic Vase,£8.00,650g,homeware,5806423927017,September 2017 06,38f124ff-1d42-4359-96f0-91490c9a6f26,Still_avaliable,M6-7203684r
1400,LB3D71C025,ODPMASE7V7,MX180RYSHX,WVPMHZP59U,BHPF2JTNKQ,PEPWA0NCVH,VIBLHHVPMN,H5N71TV8AY,OPSD21HN67


In [3]:
products_df['date_added'] = pd.to_datetime(products_df['date_added'], errors='coerce')

In [20]:
products_df[products_df['product_name'].isna()]

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
266,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
788,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
794,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1660,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN


In [4]:
products_df.dropna(how='all', inplace=True)

In [22]:
products_df[products_df['product_name'].isna()]

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code


In [5]:
products_df = products_df.loc[products_df['date_added'].notna()]
products_df = products_df.reset_index(drop=True)

In [6]:
try:
    products_df['product_price'] = products_df['product_price'].str.replace('£', '')
except AttributeError:
    pass

In [7]:
products_df['product_price'] = pd.to_numeric(products_df['product_price'])

In [42]:
products_df['product_price'].describe()

count    1846.000000
mean       16.013814
std        16.760812
min         0.390000
25%         4.000000
50%        12.000000
75%        22.000000
max       270.000000
Name: product_price, dtype: float64

In [8]:
products_df['EAN'] = pd.to_numeric(products_df['EAN'])

In [9]:
products_df['category'] = products_df['category'].astype('category')
products_df['removed'] = products_df['removed'].astype('category')

In [51]:
products_df['removed'].value_counts()

Still_avaliable    1752
Removed              94
Name: removed, dtype: int64

In [54]:
products_df['product_code'].describe()

count            1846
unique           1846
top       R7-3126933h
freq                1
Name: product_code, dtype: object

In [43]:
a = '9.4kg'
b = '10kg'
c = '10 x .99g 1000.99h'
d = '90.g'
e = '1 x 20.20g'
f = '70g.'
g = '    16.20oz '

In [12]:
# get just the numberic values and also identify where two numbers exist
if len(regex.findall('\d*\.?\d+', e)) > 1:
    print(regex.findall('\d*\.?\d+', e)[1])
else:
    regex.findall('\d*\.?\d+', e)

20.20


In [44]:
regex.findall('\d+\sx', e)[0].split(' ')[0]

'1'

In [27]:
#OR get the weight value + unit first and then just the weight 
weight_with_unit = regex.findall('\S\d*\.?\d*[A-Za-z]+', g)[0]
weight_value, weight_unit = [regex.search('\d*\.?\d*', weight_with_unit).group(),regex.search('[A-Za-z]+', weight_with_unit).group()]
weight_value, weight_unit

('16.20', 'oz')

In [45]:
# get the weight value, unit, multiplier, if any 
products_df['weight_only'] = products_df['weight'].apply(lambda w : regex.findall('\S\d*\.?\d*[A-Za-z]+', w)[0])
products_df['weight_multiplier'] = products_df['weight'].apply(lambda x : int(regex.findall('\d+\sx', x)[0].split(' ')[0]) if bool(regex.findall('\d+\sx', x)) == True else 1)


In [51]:
# get the weight value, unit, multiplier, if any (cont'd)
products_df['weight_value'] = products_df['weight_only'].apply(lambda x : regex.search('\d*\.?\d*', x).group())
products_df['weight_unit'] = products_df['weight_only'].apply(lambda x : regex.search('[A-Za-z]+', x).group())

In [68]:
conv_to_g_factors = {'kg' : 1000, 'g' : 1, 'ml' : 1, 'oz' : 28.3495}
type(conv_to_g_factors.get('oz')), conv_to_g_factors.get('oz')

(float, 28.3495)

In [69]:
# add conversion factor to convert kg, g, ml, oz to g vals
products_df['conv_to_g_factor'] = products_df['weight_unit'].apply(lambda unit : conv_to_g_factors.get(unit))


In [75]:
# finally multiply the columns, i.e. weight_value x multiplier x conv_factor = weight in grams
products_df['weight_grams'] = products_df['weight_value'].astype(float) * products_df['weight_multiplier'].astype(float) * products_df['conv_to_g_factor']

In [80]:
products_df.columns

Index(['product_name', 'product_price', 'weight', 'category', 'EAN',
       'date_added', 'uuid', 'removed', 'product_code', 'weight_only',
       'weight_multiplier', 'weight_value', 'weight_unit', 'conv_to_g_factor',
       'weight_grams'],
      dtype='object')

In [81]:
# drop columns no longer needed
products_df.drop(columns=['weight_only', 'weight_multiplier', 'weight_value', 'weight_unit', 'conv_to_g_factor'], inplace=True)

In [83]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1846 entries, 0 to 1845
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   product_name   1846 non-null   object        
 1   product_price  1846 non-null   float64       
 2   weight         1846 non-null   object        
 3   category       1846 non-null   category      
 4   EAN            1846 non-null   int64         
 5   date_added     1846 non-null   datetime64[ns]
 6   uuid           1846 non-null   object        
 7   removed        1846 non-null   category      
 8   product_code   1846 non-null   object        
 9   weight_grams   1846 non-null   float64       
dtypes: category(2), datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 119.6+ KB
